# train.csv 학습/검증

### 필요한 패키지 불러오기

먼저 필요한 패키지들을 불러와줍니다.

In [11]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings(action='ignore')
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

### train 데이터 불러오기

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
data = pd.read_csv('/content/drive/MyDrive/NLP/영화 리뷰 분석/dataset/train.csv') # train 데이터 불러오기
data.head() # 데이터 확인하기

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


In [14]:
data.isna().sum() # 결측치 확인

id          0
document    0
label       0
dtype: int64

간단하게 결측치 여부를 확인해보았습니다.

결측치가 존재하지 않는 데이터셋이군요!

### 학습/검증셋 분리하기

모델링 후 성능을 검증하기 위해 학습셋과 검증셋으로 train 데이터를 나눠줍니다.

In [15]:
from sklearn.model_selection import train_test_split

train, val = train_test_split(data)
train.reset_index(inplace=True) # 전처리 과정에서 데이터가 뒤섞이지 않도록 인덱스를 초기화해주었습니다.
val.reset_index(inplace=True)

In [16]:
train # train 셋 확인

,index,id,document,label
0,1763,1764,배드신이 너무 많아 어린아이가 보기에 그러함,0
1,4908,4909,해도 해도 너무하다. 스토리가 극단적이네,0
2,4987,4988,연기 드럽게 못하네 진짜 ㅋㅋㅋㅋㅋㅋ 이쁜척하려고 영화 찍냐,0
3,2805,2806,아주 기대되는 케이팝스타입니다ㅎㅎ,1
4,2290,2291,홍콩의 우울과 암울을 적절히 표현한 걸작,1
...,...,...,...,...
3745,544,545,"뭔가 낚인 기분, 차라리 19금 때고 코믹물로 갔었다면..",0
3746,3819,3820,보다가 끈 768번째 한국영화,0
3747,74,75,줄거리 이렇게밖에 못하나 진짜...내가 술취해서 발로써도 이거보단 나을듯,0
3748,4039,4040,난이런영화너무좋아! 류승수이상형으로바꼈다..이솜도넘이쁘고배경도영상도굿굿,1


### 데이터 정제하기

document 컬럼의 리뷰들을 전처리해줍니다. 현재 리뷰는 다양한 특수문자와 

가장 먼저 한글과 공백만 남긴 뒤 preprocessed라는 컬럼으로 저장하겠습니다.

In [17]:
train['preprocessed'] = train['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
train[:5] # 데이터 확인

,index,id,document,label,preprocessed
0,1763,1764,배드신이 너무 많아 어린아이가 보기에 그러함,0,배드신이 너무 많아 어린아이가 보기에 그러함
1,4908,4909,해도 해도 너무하다. 스토리가 극단적이네,0,해도 해도 너무하다 스토리가 극단적이네
2,4987,4988,연기 드럽게 못하네 진짜 ㅋㅋㅋㅋㅋㅋ 이쁜척하려고 영화 찍냐,0,연기 드럽게 못하네 진짜 ㅋㅋㅋㅋㅋㅋ 이쁜척하려고 영화 찍냐
3,2805,2806,아주 기대되는 케이팝스타입니다ㅎㅎ,1,아주 기대되는 케이팝스타입니다ㅎㅎ
4,2290,2291,홍콩의 우울과 암울을 적절히 표현한 걸작,1,홍콩의 우울과 암울을 적절히 표현한 걸작


추가적으로 혹시 모를 다중공백도 제거해줍니다.

In [18]:
train['preprocessed'] = train['preprocessed'].str.replace(" +", " ") # 다중 공백 제거
train[:5] # 데이터 확인

,index,id,document,label,preprocessed
0,1763,1764,배드신이 너무 많아 어린아이가 보기에 그러함,0,배드신이 너무 많아 어린아이가 보기에 그러함
1,4908,4909,해도 해도 너무하다. 스토리가 극단적이네,0,해도 해도 너무하다 스토리가 극단적이네
2,4987,4988,연기 드럽게 못하네 진짜 ㅋㅋㅋㅋㅋㅋ 이쁜척하려고 영화 찍냐,0,연기 드럽게 못하네 진짜 ㅋㅋㅋㅋㅋㅋ 이쁜척하려고 영화 찍냐
3,2805,2806,아주 기대되는 케이팝스타입니다ㅎㅎ,1,아주 기대되는 케이팝스타입니다ㅎㅎ
4,2290,2291,홍콩의 우울과 암울을 적절히 표현한 걸작,1,홍콩의 우울과 암울을 적절히 표현한 걸작


### 토큰화

#### 토큰화란?

토큰화(Tokenization)란 말뭉치(Corpus)를 주어진 단위(Token)로 나누는 과정을 의미합니다.

따라서 다음과 같이 크게 두 종류로 구분해볼 수 있습니다.

1. 문장 토큰화
2. 단어 토큰화

먼저 말뭉치를 문장 단위로 나누는 문장 토큰화(sentence tokenization)의 예시를 살펴보겠습니다.

In [19]:
text = "Hello, nice to meet you. What's your name? Have a nice day! See you soon." # 예시 문장을 정의합니다.

In [20]:
from nltk.tokenize import sent_tokenize

print('문장 토큰화 결과 ==>',sent_tokenize(text))

문장 토큰화 결과 ==> ['Hello, nice to meet you.', "What's your name?", 'Have a nice day!', 'See you soon.']


따로 분리 기준으로 "!", "?", "." 등을 설정할 필요 없이 문장 단위로 말뭉치가 분리됩니다!

다음으로 단어 단위로 나누는 단어 토큰화(word tokenization)의 예시를 살펴보겠습니다.

In [21]:
from nltk.tokenize import word_tokenize

print('단어 토큰화 결과 ==>', word_tokenize(text))

단어 토큰화 결과 ==> ['Hello', ',', 'nice', 'to', 'meet', 'you', '.', 'What', "'s", 'your', 'name', '?', 'Have', 'a', 'nice', 'day', '!', 'See', 'you', 'soon', '.']


단어 단위로 깔끔하게 분리되네요. "!", "." 등의 구두점도 하나의 단어로 취급되는 것을 확인할 수 있습니다.

이렇게 토큰화에 대해서 간단히 살펴보았습니다. 

#### 한국어에서의 토큰화

한국어는 영어와 달리 형태소(뜻을 가진 말의 최소 단위)간의 결합으로 어절이 이루어집니다. ex) 철수 -가 책 -을 읽- -었- -다

따라서 한국어 전용 형태소 분석기가 많이 존재하는데요, 이번에는 koNLPy라는 패키지의 Okt라는 형태소 분석기를 사용해보겠습니다.

In [22]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.1 MB/s 
     |████████████████████████████████| 448 kB 69.1 MB/s 


In [23]:
from konlpy.tag import Okt 

han_sentence = "오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ"
okt = Okt() # 인스턴스 할당
print("한국어 형태소 분석 결과(어간 추출X) ==>", okt.morphs(han_sentence, stem = False)) # 형태소 단위로 분리
print("한국어 형태소 분석 결과(어간 추출O) ==>", okt.morphs(han_sentence, stem = True)) # 형태소 단위로 분리 후 어간 추출

한국어 형태소 분석 결과(어간 추출X) ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O) ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


stem=True 설정한 두 번째 결과를 보면 "해볼까" -> "해보다", "힘내서" -> "힘내다", "됩시다" -> "되다" 와 같이 각 형태소의 어간을 추출해준 것을 확인할 수 있습니다. 

필요에 따라서 선택적으로 사용하면 좋을 것 같네요!

이제 영화 리뷰 데이터에서 같은 방법으로 형태소 분석을 해볼까요?

In [24]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in train['preprocessed']: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
train["tokenized_stem"] = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,index,id,document,label,preprocessed,tokenized_stem
0,1763,1764,배드신이 너무 많아 어린아이가 보기에 그러함,0,배드신이 너무 많아 어린아이가 보기에 그러함,배드 신 이 너무 많다 어리다 보기 에 그러하다
1,4908,4909,해도 해도 너무하다. 스토리가 극단적이네,0,해도 해도 너무하다 스토리가 극단적이네,해도 해도 너무하다 스토리 가 극단 적 이네
2,4987,4988,연기 드럽게 못하네 진짜 ㅋㅋㅋㅋㅋㅋ 이쁜척하려고 영화 찍냐,0,연기 드럽게 못하네 진짜 ㅋㅋㅋㅋㅋㅋ 이쁜척하려고 영화 찍냐,연기 드럽다 못 하다 진짜 ㅋㅋㅋㅋㅋㅋ 이쁘다 척 하 려고 영화 찍다
3,2805,2806,아주 기대되는 케이팝스타입니다ㅎㅎ,1,아주 기대되는 케이팝스타입니다ㅎㅎ,아주 기대 되다 케이팝스타 이다 ㅎㅎ
4,2290,2291,홍콩의 우울과 암울을 적절히 표현한 걸작,1,홍콩의 우울과 암울을 적절히 표현한 걸작,홍콩 의 우울 과 암 울 을 적절하다 표현 한 걸작


형태소 분석 및 어간 추출을 완료한 "tokenized" 컬럼이 추가되었습니다!

### 품사 태깅(POS Tagging)

자 이제 다음 단계인 품사 태깅에 대해 알아봅시다.

품사 태깅이란 주어진 텍스트를 형태소 단위로 나눈 뒤, 각 형태소에 해당 품사를 태깅하여 리스트화 하는 것입니다.

In [25]:
# 품사 태깅
print(okt.pos("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

[('오늘', 'Noun'), ('도', 'Josa'), ('열심히', 'Adverb'), ('재밌는', 'Adjective'), ('코딩', 'Noun'), ('을', 'Josa'), ('해볼까', 'Verb'), ('?', 'Punctuation'), ('같이', 'Adverb'), ('힘내서', 'Verb'), ('자연어', 'Noun'), ('처리', 'Noun'), ('고수', 'Noun'), ('들', 'Suffix'), ('이', 'Josa'), ('됩시다', 'Verb'), ('!', 'Punctuation'), ('ㅎㅎ', 'KoreanParticle')]


In [26]:
print(okt.pos("이것은 1점이 아니다 11점을 주고싶은 내 간절한 마음이다"))

[('이', 'Determiner'), ('것', 'Noun'), ('은', 'Josa'), ('1', 'Number'), ('점', 'Noun'), ('이', 'Josa'), ('아니다', 'Adjective'), ('11', 'Number'), ('점', 'Noun'), ('을', 'Josa'), ('주고싶은', 'Verb'), ('내', 'Noun'), ('간절한', 'Adjective'), ('마음', 'Noun'), ('이다', 'Josa')]


한 예시를 살펴보니 Noun, Josa, Adverb, Adjective, Verb, Suffix, Punctuation, KoreanParticle 등으로 품사가 태깅되어 하나의 리스트 형태로 반환됩니다

태그되는 품사의 종류는 더 다양하니 직접 다양한 문장을 이용해 공부해보세요!

okt의 pos 함수만을 이용해 토큰화와 품사 태깅을 한번에 할 수 있습니다.

품사 태깅은 꼭 필요한 품사(ex. 명사)를 추출할 때 유용하게 쓰일 수 있겠네요.

바로 명사를 추출해볼까요?

In [27]:
print(okt.nouns("오늘도 열심히 재밌는 코딩을 해볼까? 같이 힘내서 자연어 처리 고수들이 됩시다! ㅎㅎ"))

['오늘', '코딩', '자연어', '처리', '고수']


okt의 nouns 함수를 사용해 간편하게 명사만을 추출하였습니다!

이렇게 간단하게 명사만을 추출할수도 있지만, 영화 리뷰는 명사만으로 긍/부정을 판단하기 어렵기 때문에 

우선 임의로 명사, 동사, 형용사, 부사를 추출하여 사용해보겠습니다. (이 부분은 직접 고민하고 수정해보세요!)

In [28]:
main_pos = [] # 데이터프레임의 새 컬럼이 될 리스트
for sentence in train['document']: # 리뷰들을 하나씩 가져옵니다
    pos = okt.pos(sentence) # 형태소 분석을 진행하고 해당 리스트를 pos라는 변수로 받습니다
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")] # 가져오고자 하는 품사에 해당하면 해당 형태소를 main_words 리스트에 추가합니다.
    main_words_str = " ".join(main_words) # main_words 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    main_pos.append(main_words_str) # 선택한 형태소들로 이루어진 문자열을 최종 리스트에 추가해줍니다
train["main_pos"] = pd.DataFrame(main_pos) # 리스트를 데이터프레임으로 변환해 main_pos라는 컬럼명으로 추가해줍니다.

train.head() # 데이터 확인

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,1763,1764,배드신이 너무 많아 어린아이가 보기에 그러함,0,배드신이 너무 많아 어린아이가 보기에 그러함,배드 신 이 너무 많다 어리다 보기 에 그러하다,배드 신 너무 많아 어린아이가 보기 그러함
1,4908,4909,해도 해도 너무하다. 스토리가 극단적이네,0,해도 해도 너무하다 스토리가 극단적이네,해도 해도 너무하다 스토리 가 극단 적 이네,해도 해도 너무하다 스토리 극단
2,4987,4988,연기 드럽게 못하네 진짜 ㅋㅋㅋㅋㅋㅋ 이쁜척하려고 영화 찍냐,0,연기 드럽게 못하네 진짜 ㅋㅋㅋㅋㅋㅋ 이쁜척하려고 영화 찍냐,연기 드럽다 못 하다 진짜 ㅋㅋㅋㅋㅋㅋ 이쁘다 척 하 려고 영화 찍다,연기 드럽게 하네 진짜 이쁜 척 려고 영화 찍냐
3,2805,2806,아주 기대되는 케이팝스타입니다ㅎㅎ,1,아주 기대되는 케이팝스타입니다ㅎㅎ,아주 기대 되다 케이팝스타 이다 ㅎㅎ,아주 기대 되는 케이팝스타 입니다
4,2290,2291,홍콩의 우울과 암울을 적절히 표현한 걸작,1,홍콩의 우울과 암울을 적절히 표현한 걸작,홍콩 의 우울 과 암 울 을 적절하다 표현 한 걸작,홍콩 우울 울 적절히 표현 걸작


자 이제 총 세 개의 전처리된 컬럼이 생성되었습니다!

어떤 전처리 과정이 성능이 가장 잘 나올지는 미지수입니다. 직접 다양한 시도를 통해 최적의 전처리 프로세스를 찾아보세요~

이번 베이스라인에서는 마지막에 생성한 main_pos 컬럼을 이용해 모델 학습을 진행하겠습니다.

### 벡터화

CountVectorizer를 사용하였습니다. 

이 부분에 대한 설명은 첫 번째 베이스라인에 자세히 나와있으니 넘어가겠습니다.

In [29]:
X_train = train.main_pos #training 데이터에서 문서 추출
y_train = train.label #training 데이터에서 라벨 추출

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer() #countvectorizer 생성
vectorizer.fit(X_train) # countvectorizer 학습
X_train_vec = vectorizer.transform(X_train) # transform

### 모델 학습

In [31]:
#모델 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

models = [
    ("LR", LogisticRegression()),
    ("NB", BernoulliNB()),
    ("SGD", SGDClassifier(random_state=42, n_jobs=-1)),
    ("rfc", RandomForestClassifier(random_state=42, n_jobs=-1)),
    ("SVC", SVC(random_state=42)),
    ("ada", AdaBoostClassifier(random_state=42))
    ] #객체에 모델 할당

model = [(name, model) for name, model in models]
print(model)

lr_model = models[0][1]
nb_model = models[1][1]
sgd_model = models[2][1]
rfc_model = models[3][1]
svc_model = models[4][1]
ada_model = models[5][1]

[('LR', LogisticRegression()), ('NB', BernoulliNB()), ('SGD', SGDClassifier(n_jobs=-1, random_state=42)), ('rfc', RandomForestClassifier(n_jobs=-1, random_state=42)), ('SVC', SVC(random_state=42)), ('ada', AdaBoostClassifier(random_state=42))]


In [32]:
#모델 학습
lr_model.fit(X_train_vec, y_train)
nb_model.fit(X_train_vec, y_train)
sgd_model.fit(X_train_vec, y_train)
rfc_model.fit(X_train_vec, y_train)
svc_model.fit(X_train_vec, y_train)
ada_model.fit(X_train_vec, y_train)

AdaBoostClassifier(random_state=42)

### 검증셋으로 모델 성능 검증

먼저 검증 셋 val에 train 셋과 동일한 전처리 과정을 거쳐줍니다.

In [33]:
# preprocessed
val['preprocessed'] = val['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
val['preprocessed'] = val['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in val['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
val["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in val['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
val["main_pos"] = pd.DataFrame(main_pos)

val.head()

,index,id,document,label,preprocessed,tokenized_stem,main_pos
0,3671,3672,심각한상황에 고기구워먹고 술먹고 영화 미친듯;,0,심각한상황에 고기구워먹고 술먹고 영화 미친듯,심각하다 상황 에 고기 굽다 먹다 술 먹다 영화 미치다,심각한 상황 고기 구워 먹고 술 먹고 영화 미친듯
1,2553,2554,만화같은 이야기지만 우리 사회 재벌의 비리를 드라마로 잘 구성한 역작,1,만화같은 이야기지만 우리 사회 재벌의 비리를 드라마로 잘 구성한 역작,만화 같다 이야기 지만 우리 사회 재벌 의 비리 를 드라마 로 자다 구성 한 역작,만화 같은 이야기 우리 사회 재벌 비리 드라마 잘 구성 역작
2,3600,3601,코믹 환상 모험극. 다 좋은데 영화 찍자고 동물학대를..,0,코믹 환상 모험극 다 좋은데 영화 찍자고 동물학대를,코믹 환상 모험 극 다 좋다 영화 찍다 동물학대 를,코믹 환상 모험 다 좋은데 영화 찍자고 동물학대
3,2626,2627,진짜 시간많고 돈많은사람 한번만 보세요 열받는다. 아,0,진짜 시간많고 돈많은사람 한번만 보세요 열받는다 아,진짜 시간 많다 돈 많다 사람 한번 만 보다 열 받다 아,진짜 시간 많고 돈 많은 사람 한번 보세요 열 받는다
4,3366,3367,대만의 멜로영화치고 이렇게 재미없는건 처음..,0,대만의 멜로영화치고 이렇게 재미없는건 처음,대만 의 멜로영화 치고 이렇게 재미없다 처음,대만 멜로영화 이렇게 재미없는건 처음


학습한 모델로 예측할 X를 벡터화해줍니다.

In [34]:
X_val = val.main_pos #validation 데이터에서 전처리된 문서 추출
y_val = val.label #validation 데이터에서 라벨 추출

X_val_vec = vectorizer.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform

## 1. LogisticRegression

In [35]:
from sklearn.metrics import accuracy_score
#run model
y_pred = lr_model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

print('accuracy =', accuracy_score(y_val, y_pred)) # 정확도 확인

[0 1 0 ... 0 0 1]
accuracy = 0.8432


## 2. Naive_bayes

In [36]:
#run model
y_pred = nb_model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

print('accuracy =', accuracy_score(y_val, y_pred)) # 정확도 확인

[0 1 1 ... 0 0 1]
accuracy = 0.8384


## 3. SGDClassifier

In [37]:
#run model
y_pred = sgd_model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

print('accuracy =', accuracy_score(y_val, y_pred)) # 정확도 확인

[0 1 0 ... 1 0 0]
accuracy = 0.8264


## 4. RandomForest

In [38]:
#run model
y_pred = rfc_model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

print('accuracy =', accuracy_score(y_val, y_pred)) # 정확도 확인

[1 1 1 ... 0 0 1]
accuracy = 0.7608


## 5. AdaBoost

In [39]:
#run model
y_pred = ada_model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

print('accuracy =', accuracy_score(y_val, y_pred)) # 정확도 확인

[0 0 0 ... 0 0 0]
accuracy = 0.6952


## 6.  SVM

In [40]:
#run model
y_pred = svc_model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

print('accuracy =', accuracy_score(y_val, y_pred)) # 정확도 확인

[0 1 0 ... 0 1 1]
accuracy = 0.812


train 셋으로 학습한 모델을 사용해 validation 셋을 예측해본 결과 제일 높은 정확도는 0.844입니다!

# test.csv 분류하기

이제 본격적으로 대회에서 주어진 정답이 없는 test 데이터의 라벨을 예측해보겠습니다.

전처리 모델 학습 단계에서 test 데이터를 사용하면 data leakage로 간주되니

test.csv는 반드시 모델 학습이 끝난 뒤 예측시에만 등장해야 함에 주의해주세요!

train.csv를 이용한 전처리/학습을 test.cvs에 적용하는 것은 허용되지만, test.csv를 전처리/모델링에 직접적으로 이용하면 실격입니다.

In [41]:
test = pd.read_csv('/content/drive/MyDrive/NLP/영화 리뷰 분석/dataset/test.csv')
test

,id,document
0,1,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!
2,3,Childhood fantasy
3,4,멋있는 영화입니다. 잊을 수 없는!
4,5,너무 감동적이네요 펑펑 울었습니다
...,...,...
4995,4996,연계도 제대로 안되고 뭔가 부족한느낌이들고 찝찝한영화
4996,4997,내생에 최악의 영화중 하나 졸려서 미칠뻔....
4997,4998,지금까지 나온 드라마 중 최고였습니다. 100점 만점!
4998,4999,영화값으로 엄청난 공연을 보았다 - 이게 웬 횡재 ..


동일한 전처리 과정을 거치되, data leakage가 발생하지 않도록 주의합니다.

In [42]:
test.isna().sum()

id          0
document    0
dtype: int64

In [43]:
# preprocessed
test['preprocessed'] = test['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 한글과 공백을 제외하고 모두 제거
test['preprocessed'] = test['preprocessed'].str.replace(" +", " ") # 다중 공백 제거

# tokenized_stem
tokenized = []
for sentence in test['preprocessed']:
    tokens = okt.morphs(sentence, stem = True) # stem = True로 설정해 어간 추출을 해주었습니다
    tokenize = " ".join(tokens)
    tokenized.append(tokenize)
test["tokenized_stem"] = pd.DataFrame(tokenized)

# main_pos
main_pos = []
for sentence in test['document']:
    pos = okt.pos(sentence)
    main_words = [word_pos[0] for word_pos in pos if word_pos[1] in ("Noun", "Adverb", "Adjective", "Verb")]
    main_words_str = " ".join(main_words)
    main_pos.append(main_words_str)
test["main_pos"] = pd.DataFrame(main_pos)

test.head()

,id,document,preprocessed,tokenized_stem,main_pos
0,1,시간 때우기 좋은 영화 지루함,시간 때우기 좋은 영화 지루함,시간 때우다 좋다 영화 지루함,시간 때우기 좋은 영화 지루함
1,2,훈훈한 정이 느껴지는 영화! 가족끼리 드라마 보듯이 보면 딱~!,훈훈한 정이 느껴지는 영화 가족끼리 드라마 보듯이 보면 딱,훈훈하다 정이 느껴지다 영화 가족 끼리 드라마 보다 보다 딱,훈훈한 정이 느껴지는 영화 가족 끼리 드라마 보듯이 보면 딱
2,3,Childhood fantasy,,,
3,4,멋있는 영화입니다. 잊을 수 없는!,멋있는 영화입니다 잊을 수 없는,멋있다 영화 이다 잊다 수 없다,멋있는 영화 입니다 잊을 수 없는
4,5,너무 감동적이네요 펑펑 울었습니다,너무 감동적이네요 펑펑 울었습니다,너무 감동 적다 펑펑 울다,너무 감동 적이네요 펑펑 울었습니다


In [44]:
#Naive bayes
X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = nb_model.predict(X_test_vec)
print(pred_test)

[0 1 0 ... 1 0 1]


In [45]:
#Logestic
X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = lr_model.predict(X_test_vec)
print(pred_test)

[0 1 0 ... 1 0 1]


In [46]:
#SVM
X_test = test.main_pos
X_test_vec = vectorizer.transform(X_test)
pred_test = svc_model.predict(X_test_vec)
print(pred_test)

[0 1 0 ... 1 0 0]


In [47]:
#run model
y_pred = nb_model.predict(X_val_vec)
print(y_pred)
#0:negative 1:positive

print('accuracy =', accuracy_score(y_val, y_pred)) # 정확도 확인

[0 1 1 ... 0 0 1]
accuracy = 0.8384


# dacon 대회에 제출하기

이제 이 예측 결과를 submission.csv 파일로 만들어서 대회 페이지에 제출해보도록 합시다.

제출한 뒤 리더보드를 통해 결과를 확인합시다.

In [48]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('/content/drive/MyDrive/NLP/영화 리뷰 분석/dataset/sample_submission.csv')
submission.head()

,id,label
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [49]:
# 위에서 구한 예측값을 그대로 넣어줍니다.
submission['label'] = pred_test

# 데이터가 잘 들어갔는지 확인합니다.
submission

,id,label
0,1,0
1,2,1
2,3,0
3,4,1
4,5,1
...,...,...
4995,4996,0
4996,4997,0
4997,4998,1
4998,4999,0


In [50]:
# submission을 csv 파일로 저장합니다.
# index=False란 추가적인 id를 부여할 필요가 없다는 뜻입니다. 
# 정확한 채점을 위해 꼭 index=False를 넣어주세요.
submission.to_csv("submission_svc.csv", index=False)